In [1]:
import sys
import polars as pl
import boto3
import time
from IPython.display import display
import math
print(f"{sys.version=}")

sys.version='3.14.0 (main, Oct  7 2025, 09:34:52) [Clang 17.0.0 (clang-1700.3.19.1)]'


# Список silver S3

In [10]:
s3 = boto3.resource(
    's3',
    endpoint_url='http://localhost:9000',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False
)
storage_options = {
    "aws_endpoint_url": "http://localhost:9000", # Важно: aws_endpoint_url (иногда endpoint_url)
    "aws_access_key_id": "minioadmin",
    "aws_secret_access_key": "minioadmin",
    "aws_region": "us-east-1", # Для MinIO часто можно оставить us-east-1
    "aws_allow_http": "true",  # Разрешить HTTP (без SSL)
}

In [11]:
bucket = s3.Bucket('silver')

In [12]:
for obj in bucket.objects.all():
    print(obj)

s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=12/day=9/part-bb02b3c4-f1f2-43ab-b392-3ab4d01b31a5.parquet')


In [13]:
s3path = "s3://silver/hh/vacancies/year=2025/month=12/day=9/part-bb02b3c4-f1f2-43ab-b392-3ab4d01b31a5.parquet"

In [18]:
df_lazy = pl.scan_parquet(
    s3path,
    storage_options=storage_options,
)

In [22]:
df_lazy.select(
    pl.col('id').count().alias('id_cnt'),
    pl.col('id').unique().count().alias('unique_id_cnt')
).collect()

id_cnt,unique_id_cnt
u32,u32
148628,148628
